In [0]:
import sys, argparse
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import re
from collections import Counter
from collections import defaultdict
#from google.colab import files
#from google.colab import drive
"""uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

drive.mount('/content/drive')
with open('/content/drive/My Drive/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/drive/My\ Drive/foo.txt
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')
"""
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove.6B.zip
neg_words = []
f = open("subjclueslen1-HLTEMNLP05.tff","r")
f1 = f.readlines()
words = [line.split() for line in f1]
if words[5] == 'priorpolarity=negative':
    neg_words.append(words[2][6:]) #good list of neg words
#pos_words = []
#f = open("subjclueslen1-HLTEMNLP05.tff","r")
#f1 = f.readlines()
#words = [line.split() for line in f1]
#if words[5] == 'priorpolarity=positive':
   #pos_words.append(words[2][6:]) #good list of pos words
    '''
array_of_pos_adjs = [] #['word1=great','word1=good'...]
array_of_neg_adjs = [] #['word1=bad','word1=horrible'...]
if words[3] == 'pos1=adj' and words[5] == 'priorpolarity=positive':
    array_of_pos_adjs.append(words[2][6:])
if words[3] == 'pos1=adj' and words[5] == 'priorpolarity=negative':
  array_of_neg_adjs.append(words[2][6:])'''

In [0]:
######################################################################
## Do not edit this block of code.
## This defines the dumb features the model starts with.
######################################################################

#if the doc contains "positive" words, set the 'contains_pos_word key in feats = 1
#if the doc contains "negative" words, set the 'contains_neg_word key in feats = 1
#why are there only two keys in feats dictionary?

def dumb_featurize(text):
	feats = {}
	words = text.split(" ")

	for word in words:
		if word == "love" or word == "like" or word == "best":
			feats["contains_positive_word"] = 1
		if word == "hate" or word == "dislike" or word == "worst" or word == "awful":
			feats["contains_negative_word"] = 1

	return feats
	#returns dictionary of 2 elements (not including bias). the 2 features of the 'text'
	#are "contains_positive_word" and "contains_negative_word". Check the 'text' to see
	#if 'text' has these features, if so, make them one.



In [0]:
######################################################################
## Do not edit this block of code.
## This defines the sentiment classification class which
## loads the data and sets up the model.
######################################################################

class SentimentClassifier:

	def __init__(self, feature_method):
		self.feature_vocab = {}
		self.feature_method = feature_method


	# Read data from file
	def load_data(self, filename):
		data = []
		with open(filename, encoding="utf8") as file:
			for line in file:
				cols = line.split("\t")
				label = cols[0]
				text = cols[1].rstrip()

				data.append((label, text))
		return data

	# Featurize entire dataset
	def featurize(self, data):
		featurized_data = []
		for label, text in data:
			feats = self.feature_method(text)
			featurized_data.append((label, feats))
		return featurized_data

	# Read dataset and returned featurized representation as sparse matrix + label array
	def process(self, dataFile, training = False):
		data = self.load_data(dataFile)
		data = self.featurize(data)

		if training:			
			fid = 0
			feature_doc_count = Counter()
			for label, feats in data:
				for feat in feats:
					feature_doc_count[feat]+= 1

			for feat in feature_doc_count:
				if feature_doc_count[feat] >= MIN_FEATURE_COUNT[self.feature_method.__name__]:
					self.feature_vocab[feat] = fid
					fid += 1

		F = len(self.feature_vocab)
		D = len(data)
		X = sparse.dok_matrix((D, F))
		Y = np.zeros(D)
		for idx, (label, feats) in enumerate(data):
			for feat in feats:
				if feat in self.feature_vocab:
					X[idx, self.feature_vocab[feat]] = feats[feat]
			Y[idx] = 1 if label == "pos" else 0

		return X, Y

	def load_test(self, dataFile):
		data = self.load_data(dataFile)
		data = self.featurize(data)

		F = len(self.feature_vocab)
		D = len(data)
		X = sparse.dok_matrix((D, F))
		Y = np.zeros(D, dtype = int)
		for idx, (data_id, feats) in enumerate(data):
			# print (data_id)
			for feat in feats:
				if feat in self.feature_vocab:
					X[idx, self.feature_vocab[feat]] = feats[feat]
			Y[idx] = data_id

		return X, Y

	# Train model and evaluate on held-out data
	def evaluate(self, trainX, trainY, devX, devY):
		(D,F) = trainX.shape
		self.log_reg = linear_model.LogisticRegression(C = L2_REGULARIZATION_STRENGTH[self.feature_method.__name__])	
		self.log_reg.fit(trainX, trainY)
		training_accuracy = self.log_reg.score(trainX, trainY)
		development_accuracy = self.log_reg.score(devX, devY)
		print("Method: %s, Features: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (self.feature_method.__name__, F, training_accuracy, development_accuracy))
		

	# Predict labels for new data
	def predict(self, testX, idsX):
		predX = self.log_reg.predict(testX)

		out = open("%s_%s" % (self.feature_method.__name__, "predictions.csv"), "w", encoding="utf8")
		out.write("Id,Expected\n")
		for idx, data_id in enumerate(testX):
			out.write("%s,%s\n" % (idsX[idx], int(predX[idx])))
		out.close()

	# Write learned parameters to file
	def printWeights(self):
		out = open("%s_%s" % (self.feature_method.__name__, "weights.txt"), "w", encoding="utf8")
		reverseVocab = [None]*len(self.feature_vocab)
		for feat in self.feature_vocab:
			reverseVocab[self.feature_vocab[feat]] = feat

		out.write("%.5f\t__BIAS__\n" % self.log_reg.intercept_)
		for (weight, feat) in sorted(zip(self.log_reg.coef_[0], reverseVocab)):
			out.write("%.5f\t%s\n" % (weight, feat))
		out.close()



In [0]:
######################################################################
##You may find it helpful to change these parameters to prevent the model from overfitting 
##and achieve higher performance
######################################################################

# regularization strength to control overfitting (values closer to 0  = stronger regularization)
L2_REGULARIZATION_STRENGTH = {"dumb_featurize": 1, "fancy_featurize": .15 }
#5,5, .2
# must observe feature at least this many times in training data to include in model
MIN_FEATURE_COUNT = {"dumb_featurize": 10,  "fancy_featurize": 6 }
#5,5, 3

In [0]:
def fancy_featurize(text):
  features = {}
  text = text.lower() #string
  text = stop_words(text) #string
  features['sentence_length'] = sentence_length(text)
  text = re.sub('[^\w]+', " ", text) #get rid of punctuation, return string
  features.update(bag_of_words(text)) #returns dict
  features.update(tuple_bag_of_words(bigram(text)))
  features.update(neg_density(text.split(" "),features,neg_words))
  return features

def sentence_length(text):
  sent = re.split('\.|\?|\!',text)
  count_sent = len(sent)
  count_words = 0
  for s in sent:
    count_words += len(s.split(" "))
  return count_words/count_sent

def neg_density(word_list,features,neg_words):
  count_neg = 0
  count = Counter(word_list)
  for k,v in count.items():
    #k = k.rstrip()
    if k in neg_words:
      count_neg += v
  count_total = len(word_list)
  neg_density = count_neg/count_total
  features['neg_density'] = neg_density
  return features

def bigram(text): #returns list of tuples
  bigrams_list = []
  words = text.split(" ")
  for i in range(len(words)-1):
    bigrams_list.append((words[i], words[i+1]))
  return bigrams_list

def stop_words(text): 
  stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
  #stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
  #filtered_words = [w for w in words if w not in stop_words]
  words = text.split(" ")
  filter_words = ""
  for w in words:
    if w not in stop_words:
      filter_words += w + " "
  return filter_words #return string


In [0]:
 def tuple_bag_of_words(words):
  word_bag = {}  
  count = Counter(words) #count # times each tuple appears
  count = dict(count)
  for word in words:
    word_bag[str(word)] = count[word]
  return word_bag

In [0]:
def bag_of_words(words):
  word_bag = {}
  words = words.split(" ")
  count = Counter(words) #count # times each tuple appears
  count = dict(count)
  for word in words:
    word_bag[str(word)] = count[word]
  return word_bag

In [0]:
#this made it much worse
def count_caps(count_upper,features):
  features['count_caps'] = count_upper
  return features

def pos_density(word_list,features,pos_words):
  count_pos = 0
  count = Counter(word_list)
  for k,v in count.items():
    #k = k.rstrip()
    if k in pos_words:
      count_pos += v
  count_total = len(word_list)
  pos_density = count_pos/count_total
  features['pos_density'] = pos_density
  return features

#made it worse
def trigram(words,features):
  trigrams_list = []
  for i in range(len(words)-2):
    trigrams_list.append((words[i], words[i+1],words[i+2]))
  #bigrams = [(x, i.split()[j + 1]) for i in words for j, x in enumerate(i.split()) if j < len(i.split()) - 1] 
  #bigrams = str(zip(*[words[i:] for i in range(2)]))
  features.update(bag_of_words(trigrams_list))
  return features

def question(words,features):
  count_question = 0
  for w in words:
    if w == '\?':
      count_question += 1
  #match = re.search('\?', text)
  #if match:
    #features['question'] = 1
  #else:
    #features['question'] = 0
  features['question_num'] = count_question
  return features

#this function doesn't do anything


def caps(text,features):
  match = re.search('[A-Z\s]+',text)
  if match:
    features['all_caps'] = 1
  else:
    features['all_caps'] = 0
  return features

def count_adjs(text,features):
  count_pos_adjs = 0
  count_neg_adjs = 0
  #f = open("subjclueslen1-HLTEMNLP05.tff","r")
  #f1 = f.readlines()
  #words = [line.split() for line in f1]
  word_list = text.split(" ")
  count = Counter(word_list)
  for k,v in count.items():
    #k = k.rstrip()
    if k in array_of_pos_adjs:
      count_pos_adjs += v
    if k in array_of_neg_adjs:
      count_neg_adjs += v
  
  features['count_pos_adjs'] = count_pos_adjs
  features['count_neg_adjs'] = count_neg_adjs
  return features

#this feature was not that effective -- probably equal number of '..' in pos + neg
#didn't affect anything
def dot(words,features):
  count_dot_dot = 0
  for w in words:
    if w == '\.\.' or w == '\.\.\.':
      count_dot_dot += 1
  '''
  match1 = re.search('\.\.', text)
  match2 = re.search('\.\.\.',text)
  if match1 or match2:
    features['dot_dot'] = 1
  else:
    features['dot_dot'] = 0
  '''
  features['dot_dot'] = count_dot_dot
  return features



def create_pos_neg(word_list,features):
  f = open("subjclueslen1-HLTEMNLP05.tff","r")
  f1 = f.readlines()
  words = [line.split() for line in f1]
  if words[5] == 'priorpolarity=positive':
    #pos_words.append(words[2][6:]) 
    features['positive_word'] = 1
  if words[5] == 'priorpolarity=negative':
    features['negative_word'] = 1
  return features




#this didn't make any changes to the dev accuracy
def feat1(words,features):
  c = [tuple(words[i:i+2]) for i in range(len(words)-2)]
  for elem in c:
    if elem == ('bad', 'movie') or elem == ('worst','movie'):
      features['bad/worst_movie'] = 1
    if elem == ('good', 'movie') or elem == ('great','movie'):
      features['good/great_movie'] = 1
    else:
      features['good/great_movie'] = 0
      features['bad/worst_movie'] = 0
  return features

def feat3(text,features):
  features['char_count'] = len(text) #made it guess more incorrectly
  return features
  #features['punctuation_count'] = len("".join(i for i in text if i in string.punctuation))

def find_closest_embeddings(embedding):
    return sorted(embedded_dict.keys(), key=lambda word: numpy.linalg.norm((embedded_dict[word]- embedding)))

#makes it worse...
def punc(text,features):
  count_punc = 0
  count = Counter(text.split())

  for k,v in count.items():
    if k in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~':
      count_punc += 1
  features['punctuation_count'] = count_punc
  
  return features

def question(words,features):
  count_question = 0
  for w in words:
    if w == '\?':
      count_question += 1
  #match = re.search('\?', text)
  #if match:
    #features['question'] = 1
  #else:
    #features['question'] = 0
  features['question_num'] = count_question
  return features

In [166]:
#This code gets the train/dev/test files from github and imports them into Colab
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_1/train.txt
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_1/dev.txt
!wget https://raw.githubusercontent.com/dbamman/nlp20/master/HW_1/test.txt.zip
!unzip test.txt.zip

--2020-02-04 06:19:45--  https://raw.githubusercontent.com/dbamman/nlp20/master/HW_1/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1427184 (1.4M) [text/plain]
Saving to: ‘train.txt.7’

train.txt.7         100%[===================>]   1.36M  --.-KB/s    in 0.1s    

2020-02-04 06:19:45 (11.6 MB/s) - ‘train.txt.7’ saved [1427184/1427184]

--2020-02-04 06:19:46--  https://raw.githubusercontent.com/dbamman/nlp20/master/HW_1/dev.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1474560 (1.4M) [text/plain]
Saving to: ‘dev.txt.7’

de

In [167]:
#This cell trains two models: one on the dumb features and one on your fancy
#features.  It will store the test set predictions in a csv.
#The weights will be stored in a text file. 
#To access the files, click on the folder icon in the left sidebar.
#You can preview the files in Colab by double clicking or download the files by 
#right clicking and selecting Download.
if __name__ == "__main__":
  trainingFile = "./train.txt"
  evaluationFile = "./dev.txt"
  testFile = "./test.txt"

  for feature_method in [dumb_featurize, fancy_featurize]:
    sentiment_classifier = SentimentClassifier(feature_method)
    trainX, trainY = sentiment_classifier.process(trainingFile, training=True)
    devX, devY = sentiment_classifier.process(evaluationFile, training=False)
    testX, idsX = sentiment_classifier.load_test(testFile)
    sentiment_classifier.evaluate(trainX, trainY, devX, devY)
    sentiment_classifier.printWeights()
    sentiment_classifier.predict(testX, idsX)


Method: dumb_featurize, Features: 2, Train accuracy: 0.604, Dev accuracy: 0.611


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Method: fancy_featurize, Features: 3933, Train accuracy: 0.999, Dev accuracy: 0.828
